# Estimating Ride Duration
Here we get the user inputs
- Rideable type - Classic, Electic or Docked
- Member Type - A member or casual
- Start station
- End station

In [1]:
# import libraries for data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

import folium # for mapping and visualization
import pickle # for saving and loading model objects

import holidays # Access public holidays information for various countries

# for accessing APIs and handling API keys
import os
import requests
# import json

from geopy.distance import geodesic
# import geopandas as gpd
# import polyline


In [2]:
# Import cleaned data
station_df = pd.read_csv(r'./Data/Station Names.csv')

In [3]:
# load model
with open('RM_Model.pickle', 'rb') as to_read:
    model = pickle.load(to_read)

# load scaler
with open('Scaler.pickle', 'rb') as to_read:
    scaler = pickle.load(to_read)

### Get all station names and position
To get users desired start and end station.

In [4]:
def get_user_choice():

    options = station_df['station_name'].to_list()

    # Prompt the user to pick a station by number
    while True:
        try:
            start_choice = int(input("Enter the number of your chosen start station (1-846): "))
            if 1 <= start_choice <= len(options):
                start_station = options[start_choice - 1]  # Get the station name by its number
                return start_station
            else:
                print("Invalid choice. Please choose a valid number.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

In [5]:
# function that takes the station name and gives the position
def station_position():
    
    start_station = get_user_choice()
    start_lat = station_df[station_df['station_name'] == start_station]['latitude'].iloc[0].round(4)
    start_lng = station_df[station_df['station_name'] == start_station]['longitude'].iloc[0].round(4)

    # Ensure end station is different from start station
    while True:
        end_station = get_user_choice()
        if end_station == start_station:
            print("End station cannot be the same as the start station. Please choose a different end station.")
        else:
            break

    end_lat = station_df[station_df['station_name'] == end_station]['latitude'].iloc[0].round(4)
    end_lng = station_df[station_df['station_name'] == end_station]['longitude'].iloc[0].round(4)

    # Plot the map
    mapit = folium.Map(location= [(start_lat + end_lat)/2, (start_lng + end_lng)/2], tiles="cartodb positron", zoom_start=10)

    # add the start station
    folium.Marker(location= [start_lat, start_lng],
                icon = folium.Icon(color='orange'),
                popup=start_station).add_to(mapit)

    # add the end station
    folium.Marker(location= [end_lat, end_lng],
                icon = folium.Icon(color='green'),
                popup=end_station).add_to(mapit)


    return start_lat, start_lng, end_lat, end_lng, start_station, end_station, mapit

In [6]:
# get distance between stations
def get_distance():

    start_coord = (start_lat, start_lng)
    end_coord = (end_lat, end_lng)

    distance = geodesic(start_coord, end_coord).kilometers
    return distance

### Get current time period
Date, Hour, Time of Day, Day of Week, Season

In [7]:
# use the date to get other data

today = datetime.today()
hour = today.hour
month = today.month
weekday = today.weekday()

# time of day
def day_time():
    if 0 <= hour < 6:
        return 0 #'Early Morning'
    elif 6 <= hour < 12:
        return 1 #'Morning'
    elif 12 <= hour < 18:
        return 2 #'Afternoon'
    elif 18 <= hour < 21:
        return 3 #'Evening'
    elif 21 <= hour < 24:
        return 4 #'Night'

# get season 
def get_season():
    if month in [12, 1, 2]:  # December, January, February
        return 1 #'Winter'
    elif month in [3, 4, 5]:  # March, April, May
        return 2 #'Spring'
    elif month in [6, 7, 8]:  # June, July, August
        return 3 #'Summer'
    else:  # September, October, November
        return 4 #'Fall'

In [8]:
# check if current date is a holiday
def check_holiday():
    if datetime.today().date() in holidays.US():
        return 1
    else:
        return 0

### Current Weather Information

In [9]:
# get weather information
def get_weather():

    lat = station_df['latitude'].mean()
    lon = station_df['longitude'].mean()
    API_Key = os.environ.get("Open Weather")

    # OpenWeatherMap API URL for current weather
    url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={API_Key}&exclude=minutely,hourly,daily&units=metric'

    response = requests.get(url)
    data = response.json()

    # Check if the response was successful
    if response.status_code == 200:
        # Extract relevant data
        temperature = data['current']['temp']
        humidity = data['current']['humidity']
        wind_speed = data['current']['wind_speed']

        weather_condition = data['current']['weather'][0]['description']
        icon_mapping = {
            'clear sky': 2,
            'few clouds': 1,
            'scattered clouds': 1,
            'broken clouds': 0,
            'shower rain': 7,
            'rain': 4,
            'thunderstorm': 4,
            'snow': 5,
            'mist': 6
        }
        icon = icon_mapping.get(weather_condition, -1)  # Default to -1 if condition is unrecognized

    else:
        print("Failed to retrieve data:", data.get("message", "Error"))

    return temperature, humidity, wind_speed, icon

### Get user's desired bike type.

In [10]:
def get_bike():
    while True:
        try:
            response = int(input("What bike type: 0. Classic  1. Electric 2. Docked [Select a number] "))
            if response in [0, 1, 2]:
                    return response
            else:
                print("Invalid choice. Please enter 0, 1, or 2.")
        except ValueError:
            print("Invalid input. Please enter a numeric value (0, 1, or 2).")

### Member Type

In [11]:
def member_casual():
    while True:
        try:
            response = int(input("Pick member type: 0. Casual 1. Member"))
            if response in [0,1]:
                    return response
            else:
                print("Invalid choice. Please enter 0 or 1.")
        except ValueError:
            print("Invalid input. Please enter a numeric value (0 or 1).")

### Get user's input and reutrn the predicted ride duration.

In [12]:
member_type = member_casual()

station = station_position()
start_lat = station[0]
start_lng = station[1]
end_lat = station[2]
end_lng = station[3]
start_station = station[4]
end_station = station[5]
mapplot = station[6]

rideable_type = get_bike()

distance = get_distance()
today = datetime.today()
hour = today.hour
month = today.month
weekday = today.weekday()
time_of_day = day_time()
season = get_season()
holiday = check_holiday()

w_data = get_weather()
temp = w_data[0]
humidity = w_data[1]
windspeed = w_data[2]
weather_condition = w_data[3]

new_data = [[
    rideable_type, start_lat, start_lng, end_lat, end_lng, member_type, weekday, month,hour,
    time_of_day, season, holiday, weather_condition, temp, humidity, windspeed, distance
]]

# scale features
fitted = scaler.transform(new_data)

# predict on new data
prediction = model.predict(fitted)[0]

### Use the predicted value to calculate the revenue from the trip

In [13]:
# Pricing schema
# - Members:
#     - Unlocking Fee = $0
#     - Classic Bike  = $0.17/minute
#     - Electric Bike = $0.17/minute
#     - Docked Bike = $0.20/minute

# - Casual:
#     - Unlocking Fee = $1.35
#     - Classic Bike  = $0.20/minute
#     - Electric Bike = $0.35/minute
#     - Docked Bike = $0.40/minute

def calc_revenue(prediction):

    # per minute
    min = prediction/60
    # for members
    if member_type == 1:
        
        # classic bike rev
        if rideable_type == 0 or rideable_type == 1:
            revenue = min* 0.17
        else:
            revenue = min * 0.2

    elif member_type == 0:
        unlocking_fee = 1.35 # charge unlocking fee for casual riders
        if rideable_type == 0:
            revenue = (min * 0.2) + unlocking_fee
        elif rideable_type == 1:
            revenue = (min * 0.35) + unlocking_fee
        else: # docked bike
            revenue = (min * 0.40) + unlocking_fee

    return f"${revenue.round(2)}"

In [14]:
# Convert the duration to minutes

def sec_to_min(time):

    min = round(time/60)

    # format it
    return f"{int(min)} MIN"

In [16]:
print(f'Distance Between Stations: {round(distance, 2)} KM')
print(f'Estimated Trip Time: {sec_to_min(prediction)}')
print(f'Estimated Cost: {calc_revenue(prediction)}')

# show the start and end stations
mapplot


Distance Between Stations: 15.47 KM
Estimated Trip Time: 41 MIN
Estimated Cost: $7.03
